[INDEX](Kubernetes.ipynb)  [NEXT](6.tools.ipynb)

<img align="left" src="https://avatars1.githubusercontent.com/u/13629408?s=400&v=4" width="60" />
## 5. Maintenance Operations
<!-- <br/><br/><br/><br/> -->
<pre style="align:center" >
In this section we look at practical aspects of deploying applications to a Kubernetes cluster.

What happens when we want to upgrade the current version of the application, how we can we be sure that the upgrade has been successful and how to rollback to the previous version if needed</pre>

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @containous </td></tr></table>
    


We will look at
- Rolling upgrade
- Readystatus and health checks
- Rollback
- Node cordoning

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @containous </td></tr></table>
    

In [1]:
. .shell.functions

Loading bash helper functions, new functions added:
-- checkPresent 
-- checkState 
-- cont 
-- untilNotPresent 
-- untilNotState 
-- untilPresent 
-- untilState 


#### Let's first review our current deployment '*k8s-demo*'

In [2]:
kubectl get deploy k8s-demo

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
k8s-demo   3         3         3            3           50m


In [3]:
kubectl describe deploy k8s-demo

Name:                   k8s-demo
Namespace:              default
CreationTimestamp:      Mon, 12 Mar 2018 11:11:19 +0000
Labels:                 app=k8s-demo
Annotations:            deployment.kubernetes.io/revision=1
Selector:               app=k8s-demo
Replicas:               3 desired | 3 updated | 3 total | 3 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  1 max unavailable, 1 max surge
Pod Template:
  Labels:  app=k8s-demo
  Containers:
   k8s-demo:
    Image:        mjbright/k8s-demo:1
    Port:         8080/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   k8s-demo-6b76bd84d7 (3/3 replicas created)
Events:
  Type    Reason             Age                From                   Message
  ----    ------             ----     

We see above that we are running version 1 of our service.

In [4]:
kubectl describe deploy k8s-demo | grep Image:

    Image:        mjbright/k8s-demo:1


Once a new version is available to deploy we perform a rolling upgrade to roll out this new version:

We set the image attribute of our deployment.

We can perform a '*rollout status*' to see the upgrade as it processes

In [5]:
kubectl set image deployment k8s-demo k8s-demo=mjbright/k8s-demo:2 --record

kubectl rollout status deploy k8s-demo

deployment "k8s-demo" image updated
Waiting for rollout to finish: 1 out of 3 new replicas have been updated...
Waiting for rollout to finish: 1 out of 3 new replicas have been updated...
Waiting for rollout to finish: 2 out of 3 new replicas have been updated...
Waiting for rollout to finish: 2 out of 3 new replicas have been updated...
Waiting for rollout to finish: 1 old replicas are pending termination...
Waiting for rollout to finish: 1 old replicas are pending termination...
Waiting for rollout to finish: 1 old replicas are pending termination...
Waiting for rollout to finish: 1 old replicas are pending termination...
Waiting for rollout to finish: 2 of 3 updated replicas are available...
deployment "k8s-demo" successfully rolled out


#### Get new pods info

In [9]:
kubectl get pods -o wide

POD1=$(kubectl get pods -o wide --no-headers=true | grep k8s-demo | grep Running | head -1 | awk '{print $1;}')
POD2=$(kubectl get pods -o wide --no-headers=true | grep k8s-demo | grep Running | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pods -o wide --no-headers=true | grep k8s-demo | grep Running | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pods -o wide --no-headers=true | grep k8s-demo | grep Running | tail -1 | awk '{print $6;}')

echo POD1_IP=$POD1_IP
echo POD2_IP=$POD2_IP

NAME                        READY     STATUS    RESTARTS   AGE       IP            NODE
k8s-demo-6f66d87f98-5475d   1/1       Running   0          1m        10.192.3.27   kube-node-2
k8s-demo-6f66d87f98-m7t9s   1/1       Running   0          1m        10.192.3.28   kube-node-2
k8s-demo-6f66d87f98-txnt8   1/1       Running   0          1m        10.192.2.20   kube-node-1
POD1_IP=10.192.3.27
POD2_IP=10.192.2.20


### Observing the new version

We can now access our application to verify that the deployment has been upgraded.

In [10]:
curl $POD1_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

In [11]:
curl $POD2_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

#### Blue-green upgrades

Note that if we performed these tests during the upgrade we might see the old "blue" icon and the new "red" icon whilst some pods have been upgraded and others not.


[INDEX](Kubernetes.ipynb)  [NEXT](6.tools.ipynb)